## Setting Environment Variables

In [20]:
import os 
import sys
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

## Create a DataFrame

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("Window Functions")\
        .getOrCreate()

In [30]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100), \
    ("Ramesh", "Finance", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



## Window Ranking Functions

### row_number

In [23]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("Department").orderBy("salary")
df.withColumn("row_number", row_number().over(windowSpec)) \
    .show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         2|
|       Robert|     Sales|  4100|         3|
|         Saif|     Sales|  4100|         4|
|      Michael|     Sales|  4600|         5|
+-------------+----------+------+----------+



In [24]:
# Order them in descending
from pyspark.sql.functions import row_number, desc
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("Department").orderBy(desc("salary"))
df.withColumn("row_number", row_number().over(windowSpec)) \
    .show()

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|          Jen|   Finance|  3900|         1|
|        Scott|   Finance|  3300|         2|
|        Maria|   Finance|  3000|         3|
|         Jeff| Marketing|  3000|         1|
|        Kumar| Marketing|  2000|         2|
|      Michael|     Sales|  4600|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         3|
|        James|     Sales|  3000|         4|
|        James|     Sales|  3000|         5|
+-------------+----------+------+----------+



### Rank Function

In [25]:
from pyspark.sql.functions import rank
windowSpec  = Window.partitionBy("department").orderBy("salary")
df.withColumn("rank", rank().over(windowSpec))\
    .show()

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|   1|
|        Scott|   Finance|  3300|   2|
|          Jen|   Finance|  3900|   3|
|        Kumar| Marketing|  2000|   1|
|         Jeff| Marketing|  3000|   2|
|        James|     Sales|  3000|   1|
|        James|     Sales|  3000|   1|
|       Robert|     Sales|  4100|   3|
|         Saif|     Sales|  4100|   3|
|      Michael|     Sales|  4600|   5|
+-------------+----------+------+----+



### Dense Rank Function

In [26]:
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank", dense_rank().over(windowSpec))\
    .show()

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|        Maria|   Finance|  3000|         1|
|        Scott|   Finance|  3300|         2|
|          Jen|   Finance|  3900|         3|
|        Kumar| Marketing|  2000|         1|
|         Jeff| Marketing|  3000|         2|
|        James|     Sales|  3000|         1|
|        James|     Sales|  3000|         1|
|       Robert|     Sales|  4100|         2|
|         Saif|     Sales|  4100|         2|
|      Michael|     Sales|  4600|         3|
+-------------+----------+------+----------+



### Percent_Rank

In [27]:
from pyspark.sql.functions import percent_rank
df.withColumn("percent_rank", percent_rank().over(windowSpec)).show()

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|        Maria|   Finance|  3000|         0.0|
|        Scott|   Finance|  3300|         0.5|
|          Jen|   Finance|  3900|         1.0|
|        Kumar| Marketing|  2000|         0.0|
|         Jeff| Marketing|  3000|         1.0|
|        James|     Sales|  3000|         0.0|
|        James|     Sales|  3000|         0.0|
|       Robert|     Sales|  4100|         0.5|
|         Saif|     Sales|  4100|         0.5|
|      Michael|     Sales|  4600|         1.0|
+-------------+----------+------+------------+



### ntile

In [31]:
from pyspark.sql.functions import ntile
df.withColumn("ntile", ntile(3).over(windowSpec)).show()

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|        Maria|   Finance|  3000|    1|
|        Scott|   Finance|  3300|    1|
|          Jen|   Finance|  3900|    2|
|       Ramesh|   Finance|  4100|    3|
|        Kumar| Marketing|  2000|    1|
|         Jeff| Marketing|  3000|    2|
|        James|     Sales|  3000|    1|
|        James|     Sales|  3000|    1|
|       Robert|     Sales|  4100|    2|
|         Saif|     Sales|  4100|    2|
|      Michael|     Sales|  4600|    3|
+-------------+----------+------+-----+



## Window Analytic Function

### cume_dist

In [32]:
from pyspark.sql.functions import cume_dist
df.withColumn("cume_dist", cume_dist().over(windowSpec)).show()

+-------------+----------+------+---------+
|employee_name|department|salary|cume_dist|
+-------------+----------+------+---------+
|        Maria|   Finance|  3000|     0.25|
|        Scott|   Finance|  3300|      0.5|
|          Jen|   Finance|  3900|     0.75|
|       Ramesh|   Finance|  4100|      1.0|
|        Kumar| Marketing|  2000|      0.5|
|         Jeff| Marketing|  3000|      1.0|
|        James|     Sales|  3000|      0.4|
|        James|     Sales|  3000|      0.4|
|       Robert|     Sales|  4100|      0.8|
|         Saif|     Sales|  4100|      0.8|
|      Michael|     Sales|  4600|      1.0|
+-------------+----------+------+---------+



### Lag function

In [36]:
from pyspark.sql.functions import lag
df.withColumn("lag", lag("salary", 1).over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|null|
|        Scott|   Finance|  3300|3000|
|          Jen|   Finance|  3900|3300|
|       Ramesh|   Finance|  4100|3900|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|2000|
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|3000|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|4100|
|      Michael|     Sales|  4600|4100|
+-------------+----------+------+----+



### Lead Function

In [35]:
from pyspark.sql.functions import lead
df.withColumn("lead", lead("salary", 2).over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|4100|
|          Jen|   Finance|  3900|null|
|       Ramesh|   Finance|  4100|null|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|null|
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|null|
|      Michael|     Sales|  4600|null|
+-------------+----------+------+----+



## Window Aggregate Function

In [49]:
windowSpecAgg  = Window.partitionBy("department")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 
df.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()

+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|   Finance|3575.0|14300|3000|4100|
| Marketing|2500.0| 5000|2000|3000|
|     Sales|3760.0|18800|3000|4600|
+----------+------+-----+----+----+



In [52]:
from pyspark.sql.functions import col, sum, avg, max, min, count
windowSpecAgg = Window.partitionBy("department")
df.withColumn("row",row_number().over(windowSpec)) \
    .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
    .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
    .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
    .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
    .withColumn("count", count(col("salary")).over(windowSpecAgg)) \
    .where(col("row")==1).select("department","sum","avg","max","min","count") \
    .show()

+----------+-----+------+----+----+-----+
|department|  sum|   avg| max| min|count|
+----------+-----+------+----+----+-----+
|   Finance|14300|3575.0|4100|3000|    4|
| Marketing| 5000|2500.0|3000|2000|    2|
|     Sales|18800|3760.0|4600|3000|    5|
+----------+-----+------+----+----+-----+

